In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
import matplotlib.dates as mdates
import math
import pandas as pd
import numpy as np
from plotly import __version__
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode (connected=True)
cf.go_offline()
import datapackage
from Deep_Evolution_Strategy import Deep_Evolution_Strategy
from Model import Model
from Agent import Agent

### Heiken Ashi formula
The original model cannot catch the sudden drop/ raise trend in advanced. In order to fix this problem, I try to find some insight from the candlestick. Using Heiken Ashi to plot the candlestick would be more stable. So I apply this method into the model:
- Buy Signal: I found befor the price raise up, there is a signal looks like this one: The green candle with upper wick and without tail wick. So when the signal shows, the agent will buy bitcoin if it has money.
![alt text](Buy_Signal.png "Title")



*** After adding VIX index as another parameter, I found the performance with heiken ashi is poor. Therefore, I remove the heiken ashi as an indicater in the code. But just in case we will use it latter, I still preserve the def code here.


In [2]:
def HA(df):
    df['HA_Close']=(df['open']+ df['high']+ df['low']+df['close'])/4

    idx = df.index.name
    df.reset_index(inplace=True)

    for i in range(0, len(df)):
        if i == 0:
            df.set_value(i, 'HA_Open', ((df.get_value(i, 'open') + df.get_value(i, 'close')) / 2))
        else:
            df.set_value(i, 'HA_Open', ((df.get_value(i - 1, 'HA_Open') + df.get_value(i - 1, 'HA_Close')) / 2))

    if idx:
        df.set_index(idx, inplace=True)

    df['HA_High']=df[['HA_Open','HA_Close','high']].max(axis=1)
    df['HA_Low']=df[['HA_Open','HA_Close','low']].min(axis=1)
    return df

### Request the bitcoin price from Binance API

In [3]:
import time
import datetime
from binance.client import Client
import pandas as pd
import datetime
import dateutil.parser

In [4]:
train_start = datetime.date(2017,1,1)
train_end = datetime.date(2019,5,31)
test_start = datetime.date(2019,6,1)
test_end = datetime.date(2019,12,5)

train_start_unixtime = time.mktime(train_start.timetuple())
train_end_unixtime = time.mktime(train_end.timetuple())
test_start_unixtime = time.mktime(test_start.timetuple())
test_end_unixtime = time.mktime(test_end.timetuple())

In [5]:
def scrap_historical_binance_price(unit,start,end):

    api_key = 'ddeCyGpnkF2rrO0NyzFjjzFD6fWL4LDurf1e704GqkWck20V777VvxBNUSJkFGmP'
    api_secret = 'oZNSUX3f5Sltzr9KJBkNng8rA5hc14sV45ufNczM5MSRWFMmGcpXcQF6j0dZEkeU'

    client = Client(api_key, api_secret, {"verify": False, "timeout": 20})
    klines = client.get_historical_klines("BTCUSDT", unit, start, end)
    
    data = pd.DataFrame();
    tradingdate = []
    openprice = []
    highprice = []
    lowprice = []
    closeprice = [] 
    for i in range(len(klines)):   
        time = datetime.datetime.utcfromtimestamp(klines[i][0]/1000).strftime('%Y-%m-%d %H:%M:%S')
        format_time = dateutil.parser.parse(time)
        tradingdate.append(format_time)
        openprice.append(float(klines[i][1]))
        highprice.append(float(klines[i][2]))
        lowprice.append(float(klines[i][3]))
        closeprice.append(float(klines[i][4]))
    data['datetime'] = tradingdate
    data['low'] = lowprice
    data['high'] = highprice
    data['open'] = openprice
    data['close'] = closeprice
    data['datetime'] = pd.to_datetime(data['datetime'],unit='s')
    
    '''
    Get VIX index data
    '''
    data_url = 'https://datahub.io/core/finance-vix/datapackage.json'

    # to load Data Package into storage
    package = datapackage.Package(data_url)

    # to load only tabular data
    resources = package.resources
    for resource in resources:
        if resource.tabular:
            vix = pd.read_csv(resource.descriptor['path'])

#     vix = vix[3524:].reset_index(drop = True) # Start from 2018/01/02
    vix['Date'] = pd.to_datetime(vix.Date)
    vix['Date']=vix.Date + datetime.timedelta(days=1)
    vix.rename(columns = {'Date':'datetime','VIX Close': 'VIX_Close'}, inplace = True)
    
    df = pd.merge(data, vix, on='datetime', how='left').fillna(method='ffill')
    return df

In [6]:
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
pd.set_option('display.float_format',lambda x : '%.2f' % x)

start = str(int(train_start_unixtime))
end = str(int(train_end_unixtime))

train_df = scrap_historical_binance_price("1d", start, end)
HA(train_df)
train_df.head()

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. A

,index,datetime,low,high,open,close,VIX Open,VIX High,VIX Low,VIX_Close,HA_Close,HA_Open,HA_High,HA_Low
0,0,2017-08-17,4200.74,4485.39,4261.48,4285.08,11.59,12.54,11.25,11.74,4308.17,4273.28,4485.39,4200.74
1,1,2017-08-18,3938.77,4371.52,4285.08,4108.37,11.81,15.77,11.54,15.55,4175.94,4290.73,4371.52,3938.77
2,2,2017-08-19,3850.00,4184.69,4108.37,4139.98,15.38,16.04,13.32,14.26,4070.76,4233.33,4233.33,3850.00
3,3,2017-08-20,4032.62,4211.08,4120.98,4086.29,15.38,16.04,13.32,14.26,4112.74,4152.05,4211.08,4032.62
4,4,2017-08-21,3911.79,4119.62,4069.13,4016.00,15.38,16.04,13.32,14.26,4029.14,4132.39,4132.39,3911.79


In [7]:
test_start= str(int(test_start_unixtime))
test_df = str(int(test_end_unixtime))

test_df = scrap_historical_binance_price("1d", test_start, test_df)
HA(test_df)
test_df.head()

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning:

get_value is deprecated and will be removed in a future 

,index,datetime,low,high,open,close,VIX Open,VIX High,VIX Low,VIX_Close,HA_Close,HA_Open,HA_High,HA_Low
0,0,2019-06-01,8442.36,8626.00,8555.00,8544.07,19.05,19.72,18.01,18.71,8541.86,8549.53,8626.00,8442.36
1,1,2019-06-02,8524.00,8814.78,8545.10,8725.98,19.05,19.72,18.01,18.71,8652.47,8545.70,8814.78,8524.00
2,2,2019-06-03,8080.80,8800.95,8726.00,8115.82,19.05,19.72,18.01,18.71,8430.89,8599.08,8800.95,8080.80
3,3,2019-06-04,7481.02,8115.66,8115.66,7687.03,19.41,19.75,18.16,18.86,7849.84,8514.99,8514.99,7481.02
4,4,2019-06-05,7572.78,7896.70,7687.04,7776.50,18.74,18.74,16.97,16.97,7733.26,8182.41,8182.41,7572.78


In [8]:
close = train_df.close.values.tolist()
time = train_df.datetime.tolist()
HA_close = train_df.HA_Close.values.tolist()
HA_open = train_df.HA_Open.values.tolist()
HA_high = train_df.HA_High.values.tolist()
HA_low = train_df.HA_Low.values.tolist()
VIX_close = train_df.VIX_Close.values.tolist()
window_size = 51
skip = 1
l = len(close) - 1

In [9]:
model = Model(window_size, 500, 3)
'''
Agent(model, close,time, money, btc, btcPrice, bnbCoin)

For these parameters, you can choose 
    1) set initial money and let the agent to trade
    2) you buy the bitcoin in advanced, and let the agent to trade. 
       You need to input how many bitcoin you buy, and the price of the bitcoin.
       For example, if you buy 0.05 bitcoin at price $ 8000, you should set btc = 0.05, btcPrice = 400
- model
- close : bitcoin close price
- time : date time
- money : the initial investment money
- btc: the initial number of bitcoin
- btcPrice: the buying price for the initial number of bitcoin
- bnbCoin : initial bnbcoin

'''
agent = Agent(model,close,time,500, 0, 0, 3)
agent.fit(30, 10)

iter 10. reward: -660.632667
iter 20. reward: -326.876521
iter 30. reward: -184.074243


In [10]:
close = test_df.close.values.tolist()
time = test_df.datetime.tolist()
HA_close = test_df.HA_Close.values.tolist()
HA_open = test_df.HA_Open.values.tolist()
HA_high = test_df.HA_High.values.tolist()
HA_low = test_df.HA_Low.values.tolist()
VIX_close = test_df.VIX_Close.values.tolist()

In [11]:
agentTest = Agent(model,close,time,500, 0, 0, 3)
agentTest.buy(HA_close,HA_open,HA_high,HA_low,VIX_close)

time 2: buy 0.061608 units at price 8115.820000, total balance -0.374925,BNB coin 2.977941
time 32, sell 0.061608 units at price 11940.000000, investment 47.120069 %, total balance 734.673611, BNB coin 2.945488
time 33: buy 0.065916 units at price 11145.670000, total balance -0.550920,BNB coin 2.913076
time 35, sell 0.065916 units at price 11256.490000, investment 0.994287 %, total balance 740.870889, BNB coin 2.880342
time 36: buy 0.064953 units at price 11406.240000, total balance -0.555640,BNB coin 2.847656
time 39, sell 0.064953 units at price 12108.370000, investment 6.155666 %, total balance 785.330912, BNB coin 2.812959
time 42: buy 0.069157 units at price 11355.760000, total balance -0.588948,BNB coin 2.778312
time 61, sell 0.069157 units at price 10374.990000, investment -8.636762 %, total balance 716.376627,BNB coin 2.746657
Sell because of VIX
time 64: buy 0.065547 units at price 10929.230000, total balance -0.537255,BNB coin 2.715052
time 66, sell 0.065547 units at price 11